In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from imblearn.under_sampling import RandomUnderSampler, TomekLinks, EditedNearestNeighbours, NearMiss

In [3]:
#Importation des données guillaume.txt et création dataframe
data= pd.read_csv('guillaume.txt',sep=';')
data.head()

C:\Users\Admin\AppData\Local\Temp\ipykernel_15872\38653645.py:2: DtypeWarning: Columns (1,2,5,6,7,8,9,15,16,22) have mixed types. Specify dtype option on import or set low_memory=False.
  data= pd.read_csv('guillaume.txt',sep=';')


,ZIBZIN,IDAvisAutorisationCheque,FlagImpaye,Montant,DateTransaction,CodeDecision,VerifianceCPT1,VerifianceCPT2,VerifianceCPT3,D2CB,...,TauxImpNb_RB,TauxImpNB_CPM,EcartNumCheq,NbrMagasin3J,DiffDateTr1,DiffDateTr2,DiffDateTr3,CA3TRetMtt,CA3TR,Heure
0,A013010004908126703060931,78643044,0,20,2017-02-01 07:32:14,1,0,0,0,551,...,"37,186667890919111","52,076033757361408",0,1,4,4,4,20,0,27134
1,A013011306908024927155000,78643045,0,20,2017-02-01 07:43:37,1,0,0,0,551,...,"48,844716275908937","52,076033757361408",1,2,"1,7976851851851852",4,4,"28,609999999999999","8,6099999999999994",27817
2,A013010002908283134592527,78643046,0,"57,640000000000001",2017-02-01 07:47:38,1,0,0,0,549,...,"73,118279569892479","52,076033757361408",0,1,4,4,4,"57,640000000000001",0,28058
3,A011010002908105209831316,78643047,0,"54,289999999999999",2017-02-01 07:48:48,0,1,1,1,267,...,"110,05692599620494","53,554233554497365",0,1,4,4,4,"54,289999999999999",0,28128
4,A013010041908000125652029,78643048,0,"26,899999999999999",2017-02-01 08:13:27,1,0,0,0,549,...,"45,36831264567185","52,076033757361408",1,1,"1,9971064814814814",4,4,"59,149999999999999","32,25",29607


In [4]:
print(data.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4646774 entries, 0 to 4646773
Data columns (total 23 columns):
 #   Column                    Dtype 
---  ------                    ----- 
 0   ZIBZIN                    object
 1   IDAvisAutorisationCheque  object
 2   FlagImpaye                object
 3   Montant                   object
 4   DateTransaction           object
 5   CodeDecision              object
 6   VerifianceCPT1            object
 7   VerifianceCPT2            object
 8   VerifianceCPT3            object
 9   D2CB                      object
 10  ScoringFP1                object
 11  ScoringFP2                object
 12  ScoringFP3                object
 13  TauxImpNb_RB              object
 14  TauxImpNB_CPM             object
 15  EcartNumCheq              object
 16  NbrMagasin3J              object
 17  DiffDateTr1               object
 18  DiffDateTr2               object
 19  DiffDateTr3               object
 20  CA3TRetMtt                object
 21  CA3TR   

# Nettoyage des données 

Suppression de la ligne 1956360

In [5]:
data = data.drop(1956360)

Retypage des colonnes

In [6]:
data['ZIBZIN'] = data['ZIBZIN'].astype(str)

In [7]:
data['DateTransaction'] = pd.to_datetime(data['DateTransaction'], format="%Y-%m-%d %H:%M:%S")


In [8]:
entiers = ['VerifianceCPT1', 'VerifianceCPT2', 'VerifianceCPT3', 'D2CB', 'EcartNumCheq', 'NbrMagasin3J']

# Convertir les colonnes en type entier
data[entiers] = data[entiers].astype(int)

In [9]:
reels = ['Montant', 'ScoringFP1', 'ScoringFP2', 'ScoringFP3', 'TauxImpNb_RB', 'TauxImpNB_CPM', 'DiffDateTr1', 'DiffDateTr2', 'DiffDateTr3', 'CA3TRetMtt', 'CA3TR']

data[reels] = data[reels].applymap(lambda x: str(x).replace(',', '.')).astype(float)

C:\Users\Admin\AppData\Local\Temp\ipykernel_15872\3100941527.py:3: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  data[reels] = data[reels].applymap(lambda x: str(x).replace(',', '.')).astype(float)


Données manquantes: aucunes

In [10]:
print(data.isnull().sum())

ZIBZIN                      0
IDAvisAutorisationCheque    0
FlagImpaye                  0
Montant                     0
DateTransaction             0
CodeDecision                0
VerifianceCPT1              0
VerifianceCPT2              0
VerifianceCPT3              0
D2CB                        0
ScoringFP1                  0
ScoringFP2                  0
ScoringFP3                  0
TauxImpNb_RB                0
TauxImpNB_CPM               0
EcartNumCheq                0
NbrMagasin3J                0
DiffDateTr1                 0
DiffDateTr2                 0
DiffDateTr3                 0
CA3TRetMtt                  0
CA3TR                       0
Heure                       0
dtype: int64


In [11]:
print(data.isna().sum())

ZIBZIN                      0
IDAvisAutorisationCheque    0
FlagImpaye                  0
Montant                     0
DateTransaction             0
CodeDecision                0
VerifianceCPT1              0
VerifianceCPT2              0
VerifianceCPT3              0
D2CB                        0
ScoringFP1                  0
ScoringFP2                  0
ScoringFP3                  0
TauxImpNb_RB                0
TauxImpNB_CPM               0
EcartNumCheq                0
NbrMagasin3J                0
DiffDateTr1                 0
DiffDateTr2                 0
DiffDateTr3                 0
CA3TRetMtt                  0
CA3TR                       0
Heure                       0
dtype: int64


In [12]:
data.dtypes

ZIBZIN                              object
IDAvisAutorisationCheque            object
FlagImpaye                          object
Montant                            float64
DateTransaction             datetime64[ns]
CodeDecision                        object
VerifianceCPT1                       int32
VerifianceCPT2                       int32
VerifianceCPT3                       int32
D2CB                                 int32
ScoringFP1                         float64
ScoringFP2                         float64
ScoringFP3                         float64
TauxImpNb_RB                       float64
TauxImpNB_CPM                      float64
EcartNumCheq                         int32
NbrMagasin3J                         int32
DiffDateTr1                        float64
DiffDateTr2                        float64
DiffDateTr3                        float64
CA3TRetMtt                         float64
CA3TR                              float64
Heure                               object
dtype: obje

Distribution des classes

In [13]:
classe= data['FlagImpaye'].value_counts()
print(classe)

FlagImpaye
0    4584151
0      32627
1      29855
1        140
Name: count, dtype: int64


In [14]:
# Calcul du pourcentage de chaque classe
pourcentage_classe_0 = round((classe[0] / len(data)) * 100,2)
pourcentage_classe_1 = round((classe[1] / len(data)) * 100, 2)

print(f'Pourcentage de classe 0 : {pourcentage_classe_0}%')
print(f'Pourcentage de classe 1 : {pourcentage_classe_1}%')


Pourcentage de classe 0 : 98.65%
Pourcentage de classe 1 : 0.64%


# Extraction du fichier nettoyé au format .csv

In [15]:
data.to_csv('fichier_nettoye.csv', sep=';',index=False)

KeyboardInterrupt: 

# Apprentissage et test:

In [27]:
# Définition des dates limites pour les ensembles d'apprentissage et de test
train_inf = '2017-02-01'
train_sup = '2017-08-31'
test_inf = '2017-09-01'
test_sup = '2017-11-30'

Apprentissage:

In [30]:
train = data.loc[(data['DateTransaction'] >= train_inf) & (data['DateTransaction'] <= train_sup)]

In [33]:
X_train = train.drop(columns=['FlagImpaye','CodeDecision'])
y_train = train['FlagImpaye']

Test:

In [29]:
test = data.loc[(data['DateTransaction'] >= test_inf) & (data['DateTransaction'] <= test_sup)]

In [34]:
X_test = test.drop(columns=['FlagImpaye','CodeDecision'])
y_test = test['FlagImpaye']